# 1. Problem Information
- **Name:** [**Automated Loan Evaluation System**](https://platform.olimpiada-ai.ro/en/problems/42)
- **Date:** 12/02/2026
- **Type:** Regression

# 2. Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer,make_column_selector
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

# 3. Data preparation

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
print(train.shape)
train.head(5)

(40000, 20)


,customer_id,age,occupation_status,years_employed,annual_income,credit_score,credit_history_years,savings_assets,current_debt,defaults_on_file,delinquencies_last_2yrs,derogatory_marks,product_type,loan_intent,loan_amount,interest_rate,debt_to_income_ratio,loan_to_income_ratio,payment_to_income_ratio,loan_status
0,CUST116708,40,Self-Employed,14.1,77335,679,4.4,67,17026,0,0,0,Personal Loan,Education,26200,12.04,0.220,0.339,0.113,1
1,CUST109237,18,Employed,0.0,34946,639,1.2,292,2620,0,1,0,Personal Loan,Education,37400,14.95,0.075,1.070,0.357,0
2,CUST102214,29,Employed,9.4,26778,595,5.5,191,4702,1,1,0,Personal Loan,Personal,52300,15.51,0.176,1.953,0.651,0
3,CUST114496,27,Student,1.5,25624,590,5.7,414,2609,0,0,0,Personal Loan,Debt Consolidation,4800,15.91,0.102,0.187,0.062,0
4,CUST127659,18,Employed,0.0,25676,568,1.8,19,3453,1,1,1,Personal Loan,Medical,24700,15.47,0.134,0.962,0.321,0


In [3]:
train.describe().round(3)

,age,years_employed,annual_income,credit_score,credit_history_years,savings_assets,current_debt,defaults_on_file,delinquencies_last_2yrs,derogatory_marks,loan_amount,interest_rate,debt_to_income_ratio,loan_to_income_ratio,payment_to_income_ratio,loan_status
count,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000,40000.000
mean,34.919,7.448,50015.069,643.441,8.160,3606.282,14276.750,0.054,0.555,0.147,33044.025,15.499,0.286,0.703,0.234,0.550
std,11.119,7.591,32591.910,64.587,7.203,13349.066,13205.892,0.226,0.844,0.413,26189.168,4.058,0.160,0.467,0.156,0.497
min,18.000,0.000,15000.000,348.000,0.000,0.000,76.000,0.000,0.000,0.000,500.000,6.000,0.002,0.008,0.003,0.000
25%,26.000,1.300,27206.000,599.750,2.000,130.000,5588.000,0.000,0.000,0.000,12300.000,12.190,0.161,0.332,0.111,0.000
50%,34.000,4.900,41549.000,643.000,6.100,568.000,10379.500,0.000,0.000,0.000,26000.000,15.440,0.265,0.622,0.207,1.000
75%,43.000,11.500,62656.500,687.000,12.600,2264.250,18467.250,0.000,1.000,0.000,48600.000,18.860,0.389,1.013,0.338,1.000
max,70.000,39.900,250000.000,850.000,30.000,300000.000,163344.000,1.000,9.000,4.000,100000.000,23.000,0.800,2.001,0.667,1.000


# 4. Models

In [4]:
X = train.iloc[:,1:-1]
Y = train['loan_status']

In [5]:
transformer = make_column_transformer((OneHotEncoder(),make_column_selector(dtype_include=object)),remainder=StandardScaler())
pipeline = make_pipeline(transformer,RandomForestRegressor())

In [6]:
scores = cross_val_score(pipeline,X,Y,cv=4)
print(scores.mean())

0.7333876141802629


In [7]:
pipeline.fit(X,Y)
predictions = pipeline.predict(test.iloc[:,1:])

# 5. Submission

In [8]:
def AgeClassification(value):
    if value < 30:
        return 'Young'
    if value < 60:
        return 'Adult'
    return 'Senior'
task1 = test['age'].apply(AgeClassification)

In [9]:
def AgeClassification(value):
    if value < 20:
        return 'LowRisk'
    if value < 40:
        return 'MediumRisk'
    return 'HighRisk'
task2 = test['debt_to_income_ratio'].apply(AgeClassification)

In [10]:
task3 = test.current_debt + test.derogatory_marks + test.delinquencies_last_2yrs

In [11]:
df_task1 = pd.DataFrame({
    "subtaskID": [1]*len(test['customer_id']),
    "datapointID": test['customer_id'],
    "answer": task1
})

df_task2 = pd.DataFrame({
    "subtaskID": [2]*len(test['customer_id']),
    "datapointID": test['customer_id'],
    "answer": task2
})
df_task3 = pd.DataFrame({
    "subtaskID": [3]*len(test['customer_id']),
    "datapointID": test['customer_id'],
    "answer": task3
})
df_task4 = pd.DataFrame({
    "subtaskID": [4]*len(test['customer_id']),
    "datapointID": test['customer_id'],
    "answer": predictions
})
submission = pd.concat([df_task1, df_task2, df_task3, df_task4])
submission.head()

,subtaskID,datapointID,answer
0,1,CUST146767,Adult
1,1,CUST136829,Adult
2,1,CUST119409,Young
3,1,CUST112393,Young
4,1,CUST143342,Adult


In [12]:
submission.to_csv("submission.csv", index=False)